In [9]:
#csv
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import random
import csv

dataset_path = "D:/aiocup/challesh_4/rockyou.txt"

# دانلود و بارگذاری مدل GPT-2
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

# بارگذاری دیتاست پسورد (RockYou)
def load_password_dataset(file_path, num_samples=10000):
    with open(file_path, "r", encoding="latin-1") as f:
        passwords = [line.strip() for line in f.readlines()]
    return random.sample(passwords, num_samples)

# پردازش و آماده‌سازی داده‌ها
passwords = load_password_dataset("rockyou.txt")

# تابع تولید پسورد
def generate_password(model, tokenizer, length=12, min_length=8):
    input_text = "Generate a secure password:"
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    # ایجاد attention_mask
    attention_mask = torch.ones(input_ids.shape, device=input_ids.device)

    # تولید پسورد با طول مشخص
    output = model.generate(input_ids, attention_mask=attention_mask, max_length=length + len(input_ids[0]),
                            num_return_sequences=1, temperature=0.7, do_sample=True, pad_token_id=model.config.eos_token_id)
    
    password = tokenizer.decode(output[0], skip_special_tokens=True).replace(input_text, "").strip()
    
    # اطمینان از اینکه پسورد به درستی عددی و تصادفی است
    #password = ''.join([c for c in password if c.isdigit()])  # فقط اعداد را نگه می‌دارد
    filtered_passwords = [pwd for pwd in passwords if len(pwd) >= min_length and not pwd.isdigit()]
    
    # اگر پسورد کوتاهتر از طول مورد نظر بود، پسورد را دوباره تولید می‌کند
    while len(password) < length:
        password += str(random.randint(0, 9))
    
    return password[:length]

    # انتخاب نمونه تصادفی
    return random.sample(filtered_passwords, num_samples)

# پردازش دیتاست
passwords = load_password_dataset(dataset_path)

# ذخیره پسوردهای تمیز شده در فایل CSV
with open("clean_passwords.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Password"])  # نوشتن هدر
    for password in passwords:
        writer.writerow([password])  # نوشتن هر پسورد

print(f"✅ {len(passwords)} پسورد قوی ذخیره شد در clean_passwords.csv")

# تست تولید پسورد
print("🔐 نمونه پسورد:", generate_password(model, tokenizer, length=16))


✅ 10000 پسورد قوی ذخیره شد در clean_passwords.csv
🔐 نمونه پسورد: Use the password


In [ ]:
#اموزش که جواب داد
import os
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score
from transformers import (
    GPT2Tokenizer,
    GPT2ForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding
)
from datasets import Dataset, ClassLabel

# غیرفعال کردن دسترسی به GPU
os.environ["CUDA_VISIBLE_DEVICES"] = ""
torch.cuda.is_available = lambda: False

# تابع ایجاد برچسب بر اساس پیچیدگی رمز عبور
def calculate_complexity(password):
    length = len(password)
    has_upper = any(c.isupper() for c in password)
    has_digit = any(c.isdigit() for c in password)
    has_special = any(not c.isalnum() for c in password)
    
    score = 0
    if length >= 12: score += 2
    elif length >= 8: score += 1
    
    if has_upper: score += 1
    if has_digit: score += 1
    if has_special: score += 2
    
    if score >= 5: return 2    # قوی
    elif score >=3: return 1   # متوسط
    else: return 0             # ضعیف

# بارگذاری و پیش‌پردازش داده‌ها
def load_data(file_path):
    df = pd.read_csv(file_path)
    df['label'] = df['Password'].apply(calculate_complexity)
    passwords = df["Password"].astype(str).tolist()
    labels = df["label"].tolist()
    return passwords, labels

# تقسیم داده‌ها
passwords, labels = load_data("clean_passwords.csv")
train_texts, test_texts, train_labels, test_labels = train_test_split(
    passwords, labels, test_size=0.2, random_state=42
)

# ایجاد دیتاست‌ها
class_labels = ClassLabel(names=['weak', 'medium', 'strong'])

train_dataset = Dataset.from_dict({
    "text": train_texts,
    "label": train_labels
}).cast_column("label", class_labels)

test_dataset = Dataset.from_dict({
    "text": test_texts,
    "label": test_labels
}).cast_column("label", class_labels)

# تنظیمات توکنایزر با تعریف صریح padding token
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # تنظیم padding token
tokenizer.padding_side = "left"  # مهم برای مدل‌های GPT

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        max_length=32,
        padding="max_length",
        truncation=True,
        return_tensors="pt"  # اضافه کردن این خط
    )

# اعمال توکنایزر
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

# مدل و تنظیمات آموزش
model = GPT2ForSequenceClassification.from_pretrained(
    "gpt2",
    num_labels=3,
    problem_type="single_label_classification",
    pad_token_id=tokenizer.eos_token_id  # تنظیم pad_token_id در مدل
)
model.to("cpu")

training_args = TrainingArguments(
    output_dir="./password_classifier",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    no_cuda=True,
    logging_steps=50,
    save_strategy="epoch",
    load_best_model_at_end=True,
)

# تابع محاسبه متریک
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {
        "accuracy": accuracy_score(p.label_ids, preds),
        "f1": f1_score(p.label_ids, preds, average="weighted")
    }

# آموزش مدل
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    data_collator=DataCollatorWithPadding(
        tokenizer=tokenizer,
        padding="longest"  # تنظیم padding strategy
    ),
    compute_metrics=compute_metrics,
)

trainer.train()

# ارزیابی نهایی
predictions = trainer.predict(tokenized_test)
preds = np.argmax(predictions.predictions, axis=1)

print("\nClassification Report:")
print(classification_report(test_labels, preds, target_names=class_labels.names))

# ذخیره مدل
model.save_pretrained("password_classifier")
tokenizer.save_pretrained("password_classifier")

print("\n✅ آموزش کامل شد! مدل در پوشه password_classifier ذخیره شد.")

In [ ]:
#cuda اموزش
import os
import torch
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score
from transformers import (
    GPT2Tokenizer,
    GPT2ForSequenceClassification,
    Trainer,
    TrainingArguments,
    DataCollatorWithPadding
)
from datasets import Dataset, ClassLabel

# فعال‌سازی خودکار GPU در صورت موجود بودن
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# تابع ایجاد برچسب بر اساس پیچیدگی رمز عبور
def calculate_complexity(password):
    length = len(password)
    has_upper = any(c.isupper() for c in password)
    has_digit = any(c.isdigit() for c in password)
    has_special = any(not c.isalnum() for c in password)
    
    score = 0
    if length >= 12: score += 2
    elif length >= 8: score += 1
    
    if has_upper: score += 1
    if has_digit: score += 1
    if has_special: score += 2
    
    if score >= 5: return 2    # قوی
    elif score >=3: return 1   # متوسط
    else: return 0             # ضعیف

# بارگذاری و پیش‌پردازش داده‌ها
def load_data(file_path):
    df = pd.read_csv(file_path)
    df['label'] = df['Password'].apply(calculate_complexity)
    passwords = df["Password"].astype(str).tolist()
    labels = df["label"].tolist()
    return passwords, labels

# تقسیم داده‌ها
passwords, labels = load_data("clean_passwords.csv")
train_texts, test_texts, train_labels, test_labels = train_test_split(
    passwords, labels, test_size=0.2, random_state=42
)

# ایجاد دیتاست‌ها
class_labels = ClassLabel(names=['weak', 'medium', 'strong'])

train_dataset = Dataset.from_dict({
    "text": train_texts,
    "label": train_labels
}).cast_column("label", class_labels)

test_dataset = Dataset.from_dict({
    "text": test_texts,
    "label": test_labels
}).cast_column("label", class_labels)

# تنظیمات توکنایزر
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        max_length=32,
        padding="max_length",
        truncation=True,
        return_tensors="pt"
    )

# اعمال توکنایزر
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)

# مدل و تنظیمات آموزش
model = GPT2ForSequenceClassification.from_pretrained(
    "gpt2",
    num_labels=3,
    problem_type="single_label_classification",
    pad_token_id=tokenizer.eos_token_id
).to(device)  # انتقال مدل به GPU

# تنظیمات آموزش با GPU
training_args = TrainingArguments(
    output_dir="./password_classifier",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=32,  # افزایش اندازه بچ
    per_device_eval_batch_size=64,
    num_train_epochs=5,
    weight_decay=0.01,
    fp16=True,  # فعالسازی محاسبات 16 بیتی
    logging_steps=50,
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none",
)

# تابع محاسبه متریک
def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    return {
        "accuracy": accuracy_score(p.label_ids, preds),
        "f1": f1_score(p.label_ids, preds, average="weighted")
    }

# آموزش مدل با GPU
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    data_collator=DataCollatorWithPadding(
        tokenizer=tokenizer,
        padding="longest"
    ),
    compute_metrics=compute_metrics,
)

# شروع آموزش
trainer.train()

# ارزیابی نهایی
predictions = trainer.predict(tokenized_test)
preds = np.argmax(predictions.predictions, axis=1)

print("\nClassification Report:")
print(classification_report(test_labels, preds, target_names=class_labels.names))

# ذخیره مدل
model.save_pretrained("password_classifier")
tokenizer.save_pretrained("password_classifier")

print("\n✅ آموزش کامل شد! مدل در پوشه password_classifier ذخیره شد.")

In [7]:
#مرحله نهایی
import torch
import random
import string
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification

# بارگذاری مدل ذخیره‌شده
model_path = "password_classifier"
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2ForSequenceClassification.from_pretrained(model_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# تابع تولید رمز عبور تصادفی
def generate_password(length):
    characters = string.ascii_letters + string.digits + string.punctuation
    return ''.join(random.choice(characters) for _ in range(length))

# تابع ارزیابی امنیت رمز عبور
def evaluate_password(password):
    inputs = tokenizer(password, return_tensors="pt", padding=True, truncation=True, max_length=32).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    score = torch.argmax(outputs.logits, dim=1).item()
    levels = ["ضعیف", "متوسط", "قوی"]
    return levels[score]

# دریافت ورودی از کاربر
num_passwords = int(input("تعداد رمزهای عبور: "))
password_length = int(input("طول رمز عبور: "))

# تولید و ذخیره رمزهای عبور
with open("passwords.txt", "w", encoding="utf-8") as file:
    for _ in range(num_passwords):
        password = generate_password(password_length)
        security_level = evaluate_password(password)
        file.write(f"{password} - {security_level}\n")

print("✅ رمزهای عبور ایجاد و در فایل passwords.txt ذخیره شدند.")


✅ رمزهای عبور ایجاد و در فایل passwords.txt ذخیره شدند.
